# Лабораторная работа №6. Выполнил Акинин Александр, студент 431 группы

In [1]:
import numpy as np
import multiprocessing

### Функции подсчёта интеграла левыми, правыми и средними прямоугольниками

In [2]:
def left_rect_integral(func, a, b, h):
    n = int((b-a)/h)
    res = 0
    for i in range(n):
        x = a + i*h
        if np.isnan(y:=func(x)):
            y = func(x+h_epsilon)
        res += y
    return res*h

def right_rect_integral(func, a, b, h):
    n = int((b-a)/h)
    res = 0
    for i in range(1,n+1):
        x = a + i*h
        if np.isnan(y:=func(x)):
            y = func(x+h_epsilon)
        res += y
    return res*h

def middle_rect_integral(func, a, b, h):
    n = int((b-a)/h)
    res = 0
    for i in range(n):
        x = a + i*h + h/2
        if np.isnan(y:=func(x)):
            y = func(x+h_epsilon)
        res += y
    return res*h

In [3]:
integration_epsilon = 1e-4
h_epsilon = 1e-20

# func = lambda x: np.sin(x)*np.log(1+x)/(x**2 * np.sqrt(x))
def func(x): return np.sin(x)*np.log(1+x)/(x**2 * np.sqrt(x))
a, b = 0, 9

# Значения максимальной первой и второй производных нужны для 
# соответствия шага интегрирования точности интегрирования
# Технически они равны бесконечности, но в можем принять их как 
func_max_1st_deriv = 1e5 # << func'(x=h_epsilon) = ~1e30
func_max_2nd_deriv = 1e5 # << func''(x=h_epsilon) = ~1e50

h_lr = np.sqrt(integration_epsilon*2/func_max_1st_deriv)
h_m = np.sqrt(integration_epsilon*24/func_max_2nd_deriv)

true_res = 2.0840601436

In [4]:
left_res = left_rect_integral(func, a, b, h_lr)
print('Интеграл методом левых прямоугольников:', left_res)
print('Разность с точным значением:', abs(true_res-left_res))
print()

right_res = right_rect_integral(func, a, b, h_lr)
print('Интеграл методом правых прямоугольников:', right_res)
print('Разность с точным значением:', abs(true_res-right_res))
print()

middle_res = middle_rect_integral(func, a, b, h_m)
print('Интеграл методом средних прямоугольников:', middle_res)
print('Разность с точным значением:', abs(true_res-middle_res))
print()

/tmp/ipykernel_11599/576643013.py:5: RuntimeWarning: invalid value encountered in double_scalars
  def func(x): return np.sin(x)*np.log(1+x)/(x**2 * np.sqrt(x))


Интеграл методом левых прямоугольников: 2.0742940976745206
Разность с точным значением: 0.00976604592547936

Интеграл методом правых прямоугольников: 2.0742942723178937
Разность с точным значением: 0.009765871282106264

Интеграл методом средних прямоугольников: 2.076530673631369
Разность с точным значением: 0.00752946996863102



### Функция подсчёта интеграла методом средних прямоугольников с автоматическим выбором шага интегрирования

In [5]:
def rect_integral_auto_h(func, a, b, eps, debug=False):
    h = (b-a)/10
    
    curr = 1e10
    prev = curr*2
    
    while abs(curr-prev) > eps:
        h /= 2
        if debug: print(f'h = {h}, n = {int((b-a)/h)}')
        prev = curr
        curr = middle_rect_integral(func, a, b, h)
    
    return curr

In [6]:
middle_res_auto = rect_integral_auto_h(func, a, b, eps=5e-3, debug=False)
print('Интеграл методом средних прямоугольников c автоматическим выбором шага:', middle_res_auto)
print('Разность с точным значением:', abs(true_res-middle_res_auto))

Интеграл методом средних прямоугольников c автоматическим выбором шага: 2.0750935351984707
Разность с точным значением: 0.008966608401529186


### Функция параллельного подсчёта интеграла методом средних прямоугольников с автоматическим выбором шага интегрирования и варьированным шагом на отрезке интегрирования 

In [7]:
# Непараллельная версия функции подсчёта интеграла методом средних прямоугольников 
# с автоматическим выбором шага интегрирования и варьированным шагом на отрезке интегрирования
def rect_integral_auto_var_h(func, a, b, eps, n, debug=False):
    l = (b-a)/n
    res = 0
    for i in range(n):
        res += rect_integral_auto_h(func, a+i*l, a+(i+1)*l, eps/(2*np.log(n)), debug)
    return res

def rect_integral_auto_h_(args):
    return rect_integral_auto_h(*args)

# ПАРАЛЛЕЛЬНАЯ версия функции подсчёта интеграла методом средних прямоугольников 
# с автоматическим выбором шага интегрирования и варьированным шагом на отрезке интегрирования.
# Пришлось пребегнуть к параллелизации вычислений из-за того, что они 
# занимают очень много времени для данной точности.
def rect_integral_auto_var_h_multi(func, a, b, eps, n, debug=False):
    l = (b-a)/n
    
    func_args = [func for _ in range(n)]
    a_args = [a+i*l for i in range(n)]
    b_args = [a+(i+1)*l for i in range(n)]
    eps_args = [eps/(2*np.log(n)) for _ in range(n)]
    
    pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
    outputs = pool.map(rect_integral_auto_h_, zip(func_args, a_args, b_args, eps_args))
    return sum(outputs)

In [8]:
middle_res_auto_var = rect_integral_auto_var_h_multi(func, a, b, eps=1e-4, n=10000, debug=False)
print('Интеграл методом средних прямоугольников c автоматическим и варьированным выбором шага:', 
      middle_res_auto_var)
print('Разность с точным значением:', abs(true_res-middle_res_auto_var))

Интеграл методом средних прямоугольников c автоматическим и варьированным выбором шага: 2.0840485547833434
Разность с точным значением: 1.1588816656527001e-05
